# Sex correlations of CpGs in the methylation array for Cook Inlet belugas

If you use this code, please cite: 

Bors EK, .... 

This code draws largely on an older notebook that was used to compare normalization methods. 

The code here specifically determines the correlation of methylation level and age for each CpG included in the mammalian methylation array. I use only data that was normalized using the SeSAMe pipeline (Zhou et al., 2018) and filtered for detection pValues, requiring a good detection pValue in 10 or more samples. Total number of whales used for these runs is n=67.  



## Import Libraries

In [1]:
#import matplotlib for plotting 
import matplotlib.pyplot as plt
import matplotlib as mpl 

#Make fonts 'TrueType' for figures in pdf and ps (avoids font that won't work in many journals)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

#import pandas for working with dataframes 
import pandas as pd 
from pandas import DataFrame, read_csv 

#import numpy 
import numpy as np 

#csv reader 
import csv

#palettable for colors
import palettable

#import scipy packages
from scipy import stats 
import pylab 

#Enable in-line plotting 
%matplotlib inline

## Define File Paths and create dataframes

In [3]:
# Normalized Beta Values
# This is a csv file that has sample names as the columns and CpG beta values as the rows
# I have gone through the initial output and only chosen the 70 whales that are included in the calibration dataset 

#sesame_data_n69_10good_path = '/Users/elliebors/Desktop/Beluga/methylation_array/analysis/clock_building/pValue_10good_KBO_samp_69.csv'
#sesame_data_n69_10good_df = pd.read_csv(sesame_data_n69_path, sep=',', index_col=0)

sesame_data_n67_10good_path = '/Users/elliebors/Desktop/Beluga/methylation_array/analysis/clock_building/pValue_10good_KBO_samp_67.csv'
sesame_data_n67_10good_df = pd.read_csv(sesame_data_n67_10good_path, sep=',', index_col=0)

In [5]:
sesame_n67_10good_cpg_list = sesame_data_n67_10good_df.index[0:]
sesame_n67_10good_cpg_list

Index(['cg00000165', 'cg00001364', 'cg00001582', 'cg00002920', 'cg00003994',
       'cg00004555', 'cg00005112', 'cg00005271', 'cg00006213', 'cg00008998',
       ...
       'cg27705950', 'cg27706408', 'cg27723057', 'cg27724125', 'cg27725202',
       'cg27725275', 'cg27725290', 'cg27728043', 'cg27735940', 'cg27742797'],
      dtype='object', length=28875)

In [6]:
sex_vector = pd.read_csv('/Users/elliebors/Desktop/Beluga/methylation_array/analysis/clock_building/KBO_samp_67_sex_vector.csv',
                         sep=',', header=None)
sex_vector

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0


In [7]:
### FUNCTION ###
### make_correlation_list ###

# This function takes the following arguments: 
# (1) a list of the CpGs to consider, 
# (2) the dataframe with beta values from normalized data 
# (3) age vector for the samples - this could either the row of the beta dataframe that has ages or another list

# make_correlation_list will return a list of coefficients 

def make_correlation_list(cpg_list, beta_value_df, age_vector):
    #make ane empty list
    coefficient_list_name = []
    #Loop through the CpGs and calcuate the correlations; append to your correlation list
    for i in (cpg_list):
        correlation = np.corrcoef(age_vector, beta_value_df.loc[i])
        coefficient_list_name.append(correlation[0,1])
    return coefficient_list_name


###

### FUNCTION ###
### subset_correlations ###

# This function takes the following arguments: 
# (1) list of all coefficients, perhaps as calculated by the function 'make_correlation_list'
# (2) list of the cpg names that are associated with each correlation 
# (3) the minimum bound of your subset (subset is inclusive of lower bound) 
# (4) the maximum bound of the subset  (subset is exclusive of upper bound)

# Function returns a subset dataframe and a count of the number of items in the dataframe. 

def subset_correlations(coefficient_list_name, cpg_list_name, min_bound, max_bound):
    coefficient_subset_list = []
    cpg_subset_list = []
    for i in coefficient_list_name:
        if i >= min_bound and i < max_bound or i <= -min_bound and i > -max_bound:
            #print (i)
            cpg_subset_list.append(cpg_list_name[coefficient_list_name.index(i)])
            coefficient_subset_list.append(i)
    count = len(coefficient_subset_list)
    if len(coefficient_subset_list) == len(cpg_subset_list):
        print ('list lengths match and equal', len(coefficient_subset_list))
    subset_df = pd.DataFrame()
    subset_df['cpg_site'] = cpg_subset_list
    subset_df['coefficient'] = coefficient_subset_list
    return subset_df, count



In [8]:
sex_correlations = make_correlation_list(sesame_n67_10good_cpg_list, sesame_data_n67_10good_df, sex_vector)
#sex_correlations

In [10]:
sex_correlations_09_to_1_df, sex_count_09_to_1 = subset_correlations(sex_correlations, sesame_n67_10good_cpg_list, 0.9, 1.0)

list lengths match and equal 165


In [11]:
logistic_model = sex_correlations_09_to_1_df[sex_correlations_09_to_1_df['cpg_site']=='cg15451847']

logistic_model


,cpg_site,coefficient
106,cg15451847,-0.998843


In [19]:
sex_correlations_09_to_1_df['absolute_value_correlation']= sex_correlations_09_to_1_df['coefficient'].abs()

sex_correlations_09_to_1_df.to_csv('/Users/elliebors/Desktop/Beluga/methylation_array/analysis/sex_correlated_CpGs_09to1_KBO_samp_n67.csv')

sex_correlations_09_to_1_df.sort_values(by='absolute_value_correlation', ascending=False)


,cpg_site,coefficient,absolute_value_correlation
106,cg15451847,-0.998843,0.998843
4,cg00878023,0.995709,0.995709
50,cg07121495,-0.993246,0.993246
19,cg02150658,0.991137,0.991137
66,cg10723970,-0.990446,0.990446
72,cg11115577,-0.989853,0.989853
160,cg26617413,0.989613,0.989613
87,cg13102615,0.989411,0.989411
128,cg20415069,0.988231,0.988231
31,cg03779905,0.987002,0.987002


In [13]:
sex_correlations_08_to_09_df, sex_count_08_to_09 = subset_correlations(sex_correlations, sesame_n67_10good_cpg_list, 0.8, 0.9)

sex_correlations_08_to_09_df['absolute_value_correlation']= sex_correlations_08_to_09_df['coefficient'].abs()

sex_correlations_08_to_09_df.to_csv('/Users/elliebors/Desktop/Beluga/methylation_array/analysis/sex_correlated_08to09_CpGs_KBO_samp_n67.csv')

sex_correlations_08_to_09_df.sort_values(by='absolute_value_correlation', ascending=False)


list lengths match and equal 135


,cpg_site,coefficient,absolute_value_correlation
131,cg27204141,0.899436,0.899436
77,cg15635095,-0.898301,0.898301
15,cg04010303,-0.898281,0.898281
75,cg15472612,0.898169,0.898169
122,cg26140588,-0.896269,0.896269
109,cg23966182,-0.896104,0.896104
112,cg24606107,-0.895971,0.895971
31,cg06644235,0.895920,0.895920
38,cg07847209,-0.893559,0.893559
64,cg13772153,-0.893482,0.893482


In [14]:
sex_correlations_07_to_08_df, sex_count_07_to_08 = subset_correlations(sex_correlations, sesame_n67_10good_cpg_list, 0.7, 0.8)

sex_correlations_07_to_08_df['absolute_value_correlation']= sex_correlations_07_to_08_df['coefficient'].abs()

sex_correlations_07_to_08_df.to_csv('/Users/elliebors/Desktop/Beluga/methylation_array/analysis/sex_correlated_07to08_CpGs_KBO_samp_n67.csv')

sex_correlations_07_to_08_df.sort_values(by='absolute_value_correlation', ascending=False)


list lengths match and equal 98


,cpg_site,coefficient,absolute_value_correlation
67,cg19364802,0.798919,0.798919
62,cg18385808,0.797870,0.797870
30,cg08493441,-0.797025,0.797025
36,cg09572920,0.796645,0.796645
96,cg27069680,-0.796365,0.796365
29,cg08352048,-0.794755,0.794755
50,cg14512779,-0.794194,0.794194
71,cg20338328,-0.794110,0.794110
37,cg09580507,-0.794051,0.794051
2,cg00253867,-0.793130,0.793130


In [17]:
sex_correlations_06_to_07_df, sex_count_06_to_07 = subset_correlations(sex_correlations, sesame_n67_10good_cpg_list, 0.6, 0.7)

sex_correlations_06_to_07_df['absolute_value_correlation']= sex_correlations_06_to_07_df['coefficient'].abs()

sex_correlations_06_to_07_df.to_csv('/Users/elliebors/Desktop/Beluga/methylation_array/analysis/sex_correlated_06to07_CpGs_KBO_samp_n67.csv')

sex_correlations_06_to_07_df.sort_values(by='absolute_value_correlation', ascending=False)


list lengths match and equal 117


,cpg_site,coefficient,absolute_value_correlation
0,cg00211372,-0.698376,0.698376
18,cg06330379,0.697625,0.697625
39,cg10271516,-0.697503,0.697503
29,cg08415412,0.696461,0.696461
36,cg09736035,0.695986,0.695986
79,cg19068645,0.695177,0.695177
73,cg17002377,0.693606,0.693606
2,cg00697942,-0.693090,0.693090
54,cg12271854,0.692519,0.692519
94,cg22679901,0.689912,0.689912


In [18]:
sex_correlations_05_to_06_df, sex_count_05_to_06 = subset_correlations(sex_correlations, sesame_n67_10good_cpg_list, 0.5, 0.6)

sex_correlations_05_to_06_df['absolute_value_correlation']= sex_correlations_05_to_06_df['coefficient'].abs()

sex_correlations_05_to_06_df.to_csv('/Users/elliebors/Desktop/Beluga/methylation_array/analysis/sex_correlated_05to06_CpGs_KBO_samp_n67.csv')

sex_correlations_05_to_06_df.sort_values(by='absolute_value_correlation', ascending=False)


list lengths match and equal 113


,cpg_site,coefficient,absolute_value_correlation
65,cg14264409,-0.599618,0.599618
67,cg14331586,-0.599276,0.599276
9,cg02673411,-0.595990,0.595990
93,cg22519116,-0.595943,0.595943
13,cg02769235,0.595666,0.595666
103,cg23882198,-0.593621,0.593621
107,cg26864094,0.593513,0.593513
4,cg01609034,0.591722,0.591722
42,cg09610373,-0.591412,0.591412
26,cg04969475,0.591014,0.591014
